In [1]:
import os
base_path = os.path.dirname(os.getcwd())
data_path = base_path + '/data/word2vec'
print(base_path, data_path)

/opt/notebooks/Github/NLP /opt/notebooks/Github/NLP/data/word2vec


# Dome

In [4]:
from pprint import pprint
from paddlenlp import Taskflow
import paddle
# !python -m pip install paddlepaddle-gpu==0.0.0.post116 -f https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html
# !pip install paddlepaddle-gpu==0.0.0.post116 -f https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html
paddle.utils.run_check()

ModuleNotFoundError: No module named 'paddlenlp'

In [ ]:
# 分词切词
text = '中国人民。患者出现发热、头痛体温39度。未见明显发热。是否发热：是。双肾切面形态正常，左肾：7.3cm×2.9cm×3.2cm'

# 中文分词
seg_accurate = Taskflow('word_segmentation', mode='accurate')
seg_accurate(text)

tag = Taskflow("pos_tagging")
tag([text])


In [ ]:
# 依存语法分析
# 编码器部分将BiLSTM替换为ERNIE，模型准确率更高！
ddp = Taskflow("dependency_parsing", model="ddparser-ernie-1.0", prob=True, use_pos=True)

ddp_data = ddp("患者出现发热、头痛体温39度。未见明显发热。是否发热：是。双肾切面形态正常，左肾：7.3cm×2.9cm×3.2cm")
data_lists = []
for i in range(len(ddp_data[0]['word'])):
    print(tuple([ddp_data[0]['word'][i], ddp_data[0]['deprel'][i], ddp_data[0]['word'][ddp_data[0]['head'][i]-1], ddp_data[0]['postag'][i], ddp_data[0]['prob'][i]]))



### 属性

In [ ]:
# 关系抽取
schema = {'歌曲名称': ['歌手', '所属专辑']}  
ie = Taskflow('information_extraction', schema=schema)
ie.set_schema(schema)
ie('《告别了》是孙耀威在专辑爱的故事里面的歌曲')

In [ ]:
# 事件抽取
schema = {'地震触发词': ['地震强度', '时间', '震中位置', '震源深度']}  # 事件需要通过xxx触发词来选择触发词
ie = Taskflow('information_extraction', schema=schema)
ie.set_schema(schema)
ie('中国地震台网正式测定：5月16日06时08分在云南临沧市凤庆县(北纬24.34度，东经99.98度)发生3.5级地震，震源深度10千米。')


In [ ]:
# 情感倾向分类
schema = '情感倾向[正向，负向]' # 分类任务需要[]来设置分类的label
ie = Taskflow('information_extraction', schema=schema)
ie.set_schema(schema)
ie('这个产品用起来真的很流畅，我非常喜欢')


In [ ]:
# 评价抽取
schema = {'评价维度': ['观点词', '情感倾向[正向，负向]']}  # 评价抽取的schema是固定的，后续直接按照这个schema进行观点抽取
ie = Taskflow('information_extraction', schema=schema)
ie.set_schema(schema)
ie('地址不错，服务一般，设施陈旧')


In [ ]:
# 评价抽取
schema = ['情感倾向[正向，负向]']  # 评价抽取的schema是固定的，后续直接按照这个schema进行观点抽取
ie = Taskflow('information_extraction', schema=schema)
ie.set_schema(schema)
ie('地址不错，服务一般，设施陈旧')


In [ ]:
schema = {'发热[正向，负向]', '头痛[正向，负向]'}
schema = {'发热':'发热[正向，负向]', '头痛':'头痛[正向，负向]'}
ie = Taskflow('information_extraction', schema=schema, task_path='./checkpoint/model_best')
ie.set_schema(schema)
pprint(ie("患者出现发热、未见、头痛体温39度。未见明显发热。是否发热：是。双肾切面形态正常，左肾：7.3cm×2.9cm×3.2cm，右肾：7.3cm×3.2cm×3.5cm，左肾集合系统光点群未见明显分离，右肾集合系统光点群轻度分离，较大前后径约0.5cm，双肾实质内未见明显异常回声。右侧输尿管上段扩张，扩张较大内径约0.6cm，输尿管管壁稍毛糙，距肾门3.0cm处内可见一直径约0.8cm强回声光团，后伴声影，余输尿管显示不清。"))

### 信息抽取

In [ ]:
kpi_data = open(base_path+'/data/数据元指标名称.txt', 'r').readlines()
kpi_data = [i.replace('\n', '') for i in kpi_data]
len(kpi_data)

In [ ]:
import time
sta_time = time.time()
schema = ['脉象','既往手术名称','易饱','患者是否在经期内','脚跟痛','腹胀部位','咳嗽特点','声音撕哑','口粘','躯体水肿','鼻涕颜色','眼睑颜色','汗出位置','肢冷','反酸','嗳气','烦躁','手术史','输血史','糖尿病史','恶性肿瘤','高脂血症','辩证饮食','嗜好','小便颜色','多梦','眠可','褥疮','脚麻','腹鸣','手麻','颈痛','脸部浮肿','鼻涕性状','颈部不适','自汗','怕热','反复住院（长期住院）','下肢肿','舌质','耳胀','头痛','淋巴结肿大部位','饮酒史','外伤史','慢性肝病','高血压性心脏病','气味','干啰音','右脉','左脉','咽痒','咽部异物感','皮肤斑疹','皮肤出血','留置胃管','唇色','鼻涕','冠心病','盗汗','腹泻','寒战','胁痛','咽痛','眼干','吸烟史','睡眠','留置尿管','携带管道','皮肤皮疹','矢气多','脸肿','肌肤甲错','胃胀','头胀','头昏沉感','汗出量','肌肉酸痛','怕冷','眼睑浮肿','酸痛部位','精神状况','病程时间','舌体','心悸','头晕','欲饮','心慌','呕血','乏力','恶心','口臭','脑鸣','口苦','心肌梗死','脘腹','体位','形态','意识形态','咽反射减弱','眠差','纳可','腰酸','肢体颤抖','手抖','口渴','上肢水肿','鼻干','怕风','二便正常','胸痛','吞咽困难','烧心','耳鸣','痛经','过敏原名称','高血压病史','脑梗死','语言','淋巴结肿大','小便正常','里急后重','夜尿','易饥','干呕','流鼻血','涕中带血','鼻塞','眼花','肢体麻木','酸痛','流涕','肢体震颤','养老院','胸闷','咳嗽','尿急','腰痛','皮肤黄染','腹痛部位','胃痛','恶寒','过敏史','脑出血','风心病','肺源性心脏病','尿等待','小便量','咽干','皮肤晦暗','关节发热','关节疼痛','咯血','阴囊水肿','不欲饮','气促','尿痛','牙龈出血','发热','呕吐','尿频','浑浊','湿啰音','肛门灼热','腹满','咳痰量','眼前发黑','舌苔','皮肤干燥','意识障碍','澄清','形体','皮肤脱屑','口腔溃疡','大便正常','背痛','呼吸情况','双肺听诊','苔色','肢肿部位','肢肿','打嗝','痰液性状','长期卧床','舌色','肩痛','面色','肺部叩诊','皮肤发红','脸色','神志','口干','咳痰','痰色','下肢水肿','大便性状']
schema = kpi_data
# model: 
#   1、uie-base (默认)12-layers, 768-hidden, 12-heads；
#   2、uie-tiny 6-layers, 768-hidden, 12-heads； 
#   3、uie-medical-base 12-layers, 768-hidden, 12-heads
ie = Taskflow('information_extraction', schema=schema, model="uie-base")
ie.set_schema(schema)
print(time.time() - sta_time)

sta_time = time.time()
result = ie("体温39度。未见明显发热。是否发热：是。双肾切面形态正常，左肾：7.3cm×2.9cm×3.2cm，右肾：7.3cm×3.2cm×3.5cm，左肾集合系统光点群未见明显分离，右肾集合系统光点群轻度分离，较大前后径约0.5cm，双肾实质内未见明显异常回声。右侧输尿管上段扩张，扩张较大内径约0.6cm，输尿管管壁稍毛糙，距肾门3.0cm处内可见一直径约0.8cm强回声光团，后伴声影，余输尿管显示不清。")
print(time.time() - sta_time)
# print(type(result), result)
# result = pprint(result)
results = []
for r in result:
    for rk,rv in r.items():
        for rv_l in rv:
            if rv_l['probability'] >= 0.5:
                results.append([rk, rv_l['text'], rv_l['probability'], rv_l['start'], rv_l['end']])
import pandas as pd
results_df = pd.DataFrame(results, columns=['指标名称', '截句', '得分', '开始坐标', '结束坐标'])
results_df


In [ ]:
import pandas as pd
import json

pages_str = pd.read_csv('/home/aid/Github/kg/data/省中医风湿科入院记录_额外原文.txt', sep='\t', header=None, index_col=False, encoding='utf-8').values.tolist()
fw = open('/home/aid/Github/kg/data/省中医风湿科入院记录_额外原文_信息提取.txt', 'w', encoding='utf-8')
error_pages = 0
pages_id = 0
results = []
for i in pages_str[81:83]:
    pages_id += 1
    # try:
    medical_record = eval(i[0].strip())
    for k,v in medical_record.items():
        print(k, '---------:', v)
        result = ie(v)
        for r in result:
            for rk,rv in r.items():
                for rv_l in rv:
                    fw.write(str(pages_id)+'\t'+k+'\t'+rk+'\t'+rv_l['text']+'\t'+str(rv_l['probability'])+'\t'+str(rv_l['start'])+'\t'+str(rv_l['end'])+'\t'+v+'\n')
                    results.append([pages_id, k, rk, rv_l['text'], rv_l['probability'], rv_l['start'], rv_l['end'], v])
    # except:
    #     error_pages += 1
fw.close()
df = pd.DataFrame(results, columns=['pages_id','节点','指标名称','提取内容','得分','start','end','原文'])
df.to_csv('/home/aid/Github/kg/data/省中医风湿科入院记录_额外原文_信息提取.csv')
df

# UIE

## 数据转换 训练

In [ ]:
import pandas as pd
import os
print(os.getcwd())

In [ ]:
!/usr/bin/python3 /opt/notebooks/Github/NLP/models/UIE/doccano.py \
    --doccano_file /opt/notebooks/Github/NLP/data/uie_paddle/医病.json \
    --task_type "ext" \
    --save_dir /opt/notebooks/Github/NLP/data/uie_paddle \
    --splits 0.1 0.9 0

In [ ]:
!/usr/bin/python3 /opt/notebooks/Github/NLP/models/UIE/finetune.py \
    --train_path "/opt/notebooks/Github/NLP/data/uie_paddle/train.txt" \
    --dev_path "/opt/notebooks/Github/NLP/data/uie_paddle/dev.txt" \
    --save_dir "./checkpoint" \
    --learning_rate 1e-5 \
    --batch_size 8 \
    --max_seq_len 512 \
    --num_epochs 1 \
    --model "uie-base" \
    --seed 1000 \
    --logging_steps 10 \
    --valid_steps 100 \
    --device "gpu"

In [ ]:
!/usr/bin/python3 UIE/evaluate.py \
    --model_path "./checkpoint/model_best" \
    --test_path "/opt/notebooks/Github/NLP/data/uie_paddle/dev.txt" \
    --batch_size 16 \
    --max_seq_len 512

In [ ]:
from pprint import pprint
from paddlenlp import Taskflow
import pandas as pd

schema = ['肌力','呼吸节律的类型','生理反射是否存在','病理反射情况','吸烟史','饮酒史','吸毒史','冶游史','疫水接触史','疫区久居史','意识障碍','精神情况','意识障碍的分类','大便情况','黄染的部位','大血管枪击音','水冲脉','有无毛细血管搏动','收缩压（mmHg）','舒张压（mmHg）','发育情况','神志','T（℃）','R（次/分）','P（次/分）','体型状况','体位情况','表情情况','浅表淋巴结有无肿大','双侧瞳孔等大','双侧瞳孔等圆','瞳孔对光反射','外耳道有无分泌物','左侧瞳孔大小（mm）','右侧瞳孔大小（mm）','口唇情况','伸舌是否居中','颈静脉回流征','病理性杂音','肋间隙情况','呼吸音情况','移动性浊音','肠鸣音是否正常','肌张力','配偶健康情况','婚姻情况','生育情况','家族疾病名称','过敏史','过敏药物名称','输血史','手术史','外伤史','外伤原因','既往手术名称','平素健康状况','传染病史','既往疾病','既往疾病名称','具体传染病名称','预防接种史','既往疾病名称-病史年限','既往疾病名称-服用药物名称','咳痰量','咯血的颜色','咯血量','小便情况','睡眠情况','剧吐','胃痛','入院指脉氧（%）','家族疾病史']
schema = ['脉象','既往手术名称','易饱','患者是否在经期内','脚跟痛','腹胀部位','咳嗽特点','声音撕哑','口粘','躯体水肿','鼻涕颜色','眼睑颜色','汗出位置','肢冷','反酸','嗳气','烦躁','手术史','输血史','糖尿病史','恶性肿瘤','高脂血症','辩证饮食','嗜好','小便颜色','多梦','眠可','褥疮','脚麻','腹鸣','手麻','颈痛','脸部浮肿','鼻涕性状','颈部不适','自汗','怕热','反复住院（长期住院）','下肢肿','舌质','耳胀','头痛','淋巴结肿大部位','饮酒史','外伤史','慢性肝病','高血压性心脏病','气味','干啰音','右脉','左脉','咽痒','咽部异物感','皮肤斑疹','皮肤出血','留置胃管','唇色','鼻涕','冠心病','盗汗','腹泻','寒战','胁痛','咽痛','眼干','吸烟史','睡眠','留置尿管','携带管道','皮肤皮疹','矢气多','脸肿','肌肤甲错','胃胀','头胀','头昏沉感','汗出量','肌肉酸痛','怕冷','眼睑浮肿','酸痛部位','精神状况','病程时间','舌体','心悸','头晕','欲饮','心慌','呕血','乏力','恶心','口臭','脑鸣','口苦','心肌梗死','脘腹','体位','形态','意识形态','咽反射减弱','眠差','纳可','腰酸','肢体颤抖','手抖','口渴','上肢水肿','鼻干','怕风','二便正常','胸痛','吞咽困难','烧心','耳鸣','痛经','过敏原名称','高血压病史','脑梗死','语言','淋巴结肿大','小便正常','里急后重','夜尿','易饥','干呕','流鼻血','涕中带血','鼻塞','眼花','肢体麻木','酸痛','流涕','肢体震颤','养老院','胸闷','咳嗽','尿急','腰痛','皮肤黄染','腹痛部位','胃痛','恶寒','过敏史','脑出血','风心病','肺源性心脏病','尿等待','小便量','咽干','皮肤晦暗','关节发热','关节疼痛','咯血','阴囊水肿','不欲饮','气促','尿痛','牙龈出血','发热','呕吐','尿频','浑浊','湿啰音','肛门灼热','腹满','咳痰量','眼前发黑','舌苔','皮肤干燥','意识障碍','澄清','形体','皮肤脱屑','口腔溃疡','大便正常','背痛','呼吸情况','双肺听诊','苔色','肢肿部位','肢肿','打嗝','痰液性状','长期卧床','舌色','肩痛','面色','肺部叩诊','皮肤发红','脸色','神志','口干','咳痰','痰色','下肢水肿','大便性状']
# df = pd.read_csv('data/uie_paddle/医病.csv', sep=',')
# kpi_name = list(set(df['kpi_name'].values.tolist()))
# schema = kpi_name

# 设定抽取目标和定制化模型权重路径
my_ie = Taskflow("information_extraction", schema=schema, task_path='./checkpoint/model_best')
my_ie.set_schema(schema)
data = pd.read_excel(base_path + '/data/uie_paddle/茂名人民医院重症专病入院记录.xlsx').values.tolist()
pre_data = []
for i in data[:10]:
    try:
        print(i[8], '---->', my_ie(i[8]))
        pre_data.append(my_ie(i[8]))
    except:
        pass

In [ ]:
from pprint import pprint
from paddlenlp import Taskflow
import pandas as pd

schema = ['肌力','呼吸节律的类型','生理反射是否存在','病理反射情况','吸烟史','饮酒史','吸毒史','冶游史','疫水接触史','疫区久居史','意识障碍','精神情况','意识障碍的分类','大便情况','黄染的部位','大血管枪击音','水冲脉','有无毛细血管搏动','收缩压（mmHg）','舒张压（mmHg）','发育情况','神志','T（℃）','R（次/分）','P（次/分）','体型状况','体位情况','表情情况','浅表淋巴结有无肿大','双侧瞳孔等大','双侧瞳孔等圆','瞳孔对光反射','外耳道有无分泌物','左侧瞳孔大小（mm）','右侧瞳孔大小（mm）','口唇情况','伸舌是否居中','颈静脉回流征','病理性杂音','肋间隙情况','呼吸音情况','移动性浊音','肠鸣音是否正常','肌张力','配偶健康情况','婚姻情况','生育情况','家族疾病名称','过敏史','过敏药物名称','输血史','手术史','外伤史','外伤原因','既往手术名称','平素健康状况','传染病史','既往疾病','既往疾病名称','具体传染病名称','预防接种史','既往疾病名称-病史年限','既往疾病名称-服用药物名称','咳痰量','咯血的颜色','咯血量','小便情况','睡眠情况','剧吐','胃痛','入院指脉氧（%）','家族疾病史']
schema = ['脉象','既往手术名称','易饱','患者是否在经期内','脚跟痛','腹胀部位','咳嗽特点','声音撕哑','口粘','躯体水肿','鼻涕颜色','眼睑颜色','汗出位置','肢冷','反酸','嗳气','烦躁','手术史','输血史','糖尿病史','恶性肿瘤','高脂血症','辩证饮食','嗜好','小便颜色','多梦','眠可','褥疮','脚麻','腹鸣','手麻','颈痛','脸部浮肿','鼻涕性状','颈部不适','自汗','怕热','反复住院（长期住院）','下肢肿','舌质','耳胀','头痛','淋巴结肿大部位','饮酒史','外伤史','慢性肝病','高血压性心脏病','气味','干啰音','右脉','左脉','咽痒','咽部异物感','皮肤斑疹','皮肤出血','留置胃管','唇色','鼻涕','冠心病','盗汗','腹泻','寒战','胁痛','咽痛','眼干','吸烟史','睡眠','留置尿管','携带管道','皮肤皮疹','矢气多','脸肿','肌肤甲错','胃胀','头胀','头昏沉感','汗出量','肌肉酸痛','怕冷','眼睑浮肿','酸痛部位','精神状况','病程时间','舌体','心悸','头晕','欲饮','心慌','呕血','乏力','恶心','口臭','脑鸣','口苦','心肌梗死','脘腹','体位','形态','意识形态','咽反射减弱','眠差','纳可','腰酸','肢体颤抖','手抖','口渴','上肢水肿','鼻干','怕风','二便正常','胸痛','吞咽困难','烧心','耳鸣','痛经','过敏原名称','高血压病史','脑梗死','语言','淋巴结肿大','小便正常','里急后重','夜尿','易饥','干呕','流鼻血','涕中带血','鼻塞','眼花','肢体麻木','酸痛','流涕','肢体震颤','养老院','胸闷','咳嗽','尿急','腰痛','皮肤黄染','腹痛部位','胃痛','恶寒','过敏史','脑出血','风心病','肺源性心脏病','尿等待','小便量','咽干','皮肤晦暗','关节发热','关节疼痛','咯血','阴囊水肿','不欲饮','气促','尿痛','牙龈出血','发热','呕吐','尿频','浑浊','湿啰音','肛门灼热','腹满','咳痰量','眼前发黑','舌苔','皮肤干燥','意识障碍','澄清','形体','皮肤脱屑','口腔溃疡','大便正常','背痛','呼吸情况','双肺听诊','苔色','肢肿部位','肢肿','打嗝','痰液性状','长期卧床','舌色','肩痛','面色','肺部叩诊','皮肤发红','脸色','神志','口干','咳痰','痰色','下肢水肿','大便性状']
df = pd.read_csv('data/uie_paddle/医病.csv', sep=',')
kpi_name = list(set(df['kpi_name'].values.tolist()))
schema = kpi_name

my_ie = Taskflow('information_extraction', schema=schema, model="uie-base")
data = pd.read_excel((base_path + '/data/uie_paddle/茂名人民医院重症专病入院记录.xlsx').values.tolist()
pre_data = []
for i in data[:10]:
    try:
        print(i[8], '---->', my_ie(i[8]))
        pre_data.append(my_ie(i[8]))
    except:
        pass


## 自定义数据转换

In [ ]:
# {"id": 1, 
# "text": "昨天晚上十点加班打车回家58元", 
# "relations": [], 
# "entities": [
#     {"id": 0, "start_offset": 0, "end_offset": 6, "label": "时间"}, 
#     {"id": 1, "start_offset": 11, "end_offset": 12, "label": "目的地"}, 
#     {"id": 2, "start_offset": 12, "end_offset": 14, "label": "费用"}]}

In [ ]:
import pandas as pd
import re, os, json
import pymysql
print(os.getcwd())

In [ ]:
sql = 'SELECT * FROM (SELECT doc_id,kwd,result_value FROM `aid-tagging`.tagging) AS ta_a JOIN (SELECT id,doc_content FROM `aid-tagging`.document) AS ta_b ON ta_a.doc_id=ta_b.id'
connection = pymysql.connect(host='192.168.100.50', port=3306, user='root',password='Aid@pro888888', db='nlp_tagging', charset='utf8mb4')
cursor = connection.cursor()
cursor.execute(sql)
data = cursor.fetchall()
cursor.close()
connection.close()
data

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data, columns=['id', 'kwd', 'type', 'sta_en', 'content'])
df_grou = df.groupby(['id', 'kwd', 'type', 'sta_en'])['content'].apply(lambda x: x.str.cat(sep='<!>')).reset_index()
df_grou

In [ ]:
df_data = df.groupby(['id', 'kwd', 'type', 'sta_en'])['content'].apply(lambda x: x.str.cat(sep='<!>')).reset_index()
df_data

In [ ]:
df = pd.read_excel(base_path + '/data/uie_paddle/医病.xlsx')
df_data = df.groupby(['task_name','finish_nickname','second_node','kpi_code','kpi_name','label_content','kpi_value','sent'])['sent_full'].apply(lambda x: x.str.cat(sep='<!>')).reset_index()
df_data.to_csv(base_path + '/data/uie_paddle/医病.csv')
df_data = df_data.values.tolist()

rules = []
with open(base_path + '/data/uie_paddle/医病.json', 'w') as f_six:
    for i in range(len(df_data)):
        # print(df_data[i])
        # print(df_data[i][8])
        tmp_dict = {'id': i+1, 'text': df_data[i][8], 'relations':[], 'entities':[]}
        try:
            for m in re.finditer(df_data[i][5], df_data[i][8]):
                # print(m.span()[0], m.span(), m.group(), df_data[i][4])
                tmp_dict['entities'].append({'id':0, 'start_offset':m.span()[0], 'end_offset':m.span()[1], 'label':df_data[i][4]})
            f_six.write(json.dumps(tmp_dict, ensure_ascii=False)+'\n')
        except:
            print(df_data[i][5])
            print(df_data[i][8])
f_six.close()


In [ ]:
from paddlenlp.transformers import *

tokenizer = AutoTokenizer.from_pretrained('ernie-health-chinese')

# 分类任务
# model = AutoModelForSequenceClassification.from_pretrained('ernie-health-chinese')
# 序列标注任务
model = AutoModelForTokenClassification.from_pretrained('ernie-health-chinese')
# 阅读理解任务
# model = AutoModelForQuestionAnswering.from_pretrained('ernie-health-chinese')

## TokenEmbedding

In [ ]:
import paddle
from paddlenlp.embeddings import TokenEmbedding, list_embedding_name
paddle.set_device("cpu")

# 查看预训练embedding名称：
# print(list_embedding_name()) # ['w2v.baidu_encyclopedia.target.word-word.dim300']

# 初始化TokenEmbedding， 预训练embedding没下载时会自动下载并加载数据
# extended_vocab_path 扩展词表的文件名路径。词表格式为一行一个词。
# trainable 是否可训练。True表示Embedding可以更新参数，False为不可更新。
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300",
                                extended_vocab_path = data_path+'/补充词.txt',
                                trainable = True
                                )

# 切词
from paddlenlp.data import JiebaTokenizer
tokenizer = JiebaTokenizer(vocab=token_embedding.vocab)
words = tokenizer.cut("中国人民")
print(words) # ['中国人', '民']
tokens = tokenizer.encode("中国人民")
print(tokens) # [12530, 1334]

# 查看token_embedding详情
print(token_embedding)

test_token_embedding = token_embedding.search("中国")
# print('词向量：', test_token_embedding)

score = token_embedding.cosine_sim("中国", "美国")
print('计算词向量cosine相似度：', score)

#使用paddle函数计算余弦相似度
w2v = token_embedding.weight
print(paddle.nn.functional.cosine_similarity(w2v[2965],w2v[3886],axis=0))

In [ ]:
score = token_embedding.cosine_sim("错位", "紊乱")
print('计算词向量cosine相似度：', score)

In [ ]:
# 词性标注
from paddlenlp import Taskflow
import pandas as pd
import os
print(os.getcwd())


def related_words(word):
    data = pd.read_csv(data_path + '/../病历/省中医风湿科入院记录_额外原文.txt', sep='\t', header=None, index_col=False, encoding='utf-8').values.tolist()

    data_1 = []
    for i in data:
        data_1.append(i[0])

    tag = Taskflow("pos_tagging")
    word_cat = tag([word])
    
    word_list = []
    for w in word_cat:
        tag_list = tag(data_1[0])
        for i in tag_list:
            score = token_embedding.cosine_sim(w[0], i[0])
            if i[0] in "椎动脉":
                print(w, i[0], score)
            if score >= 0.51:
                word_list.append([w, i[0], score])
    word_df = None
    if len(word_list) > 1:
        word_df = pd.DataFrame(word_list, columns=['k', 'word','score'])
        word_df.sort_values(by='score',axis=0, ascending=False, inplace=True)
    
    print(word_df)
    return word_df

def cut_pos_tagging():
    data = pd.read_csv(data_path + '/dk_mainsuit_800w.txt', sep='\t', header=None, index_col=False, encoding='utf-8').values.tolist()
    
    data_1 = []
    for i in data:
        data_1.append(i[0])
        
    tag = Taskflow("pos_tagging")
    word_cat = tag(data_1)
    
    words = []
    stop_word = []
    for x in word_cat:
        for y in x:
            if y[1] != 'w':
                words.append(y[0])
            else:
                stop_word.append(y[0])
    words = list(set(words))
    stop_word = list(set(stop_word))
    fw = open(data_path + '/补充词.txt', 'w')
    for i in words:
        fw.write(i+'\n')
    fw.close()
    fw = open(data_path + '/停用词.txt', 'w')
    for i in words:
        fw.write(i+'\n')
    fw.close()
    
    return words


# related_words("BPM")
word_cat = cut_pos_tagging()
word_cat[:10]

In [ ]:
kwd = open('kwd.txt', 'r').readlines(
for i in kwd:
    print(i.strip())
    related_words(i.strip())

## Paddle词向量训练